In [1]:
%cd ..

/home/cole/Documents/dl-final-project


In [22]:
import torch
from data import get_unlabeled_set
from model.resnet import Prototype

In [23]:
batch_size = 3

if torch.cuda.is_available():
    batch_size *= torch.cuda.device_count()

unlabeled_trainset, unlabeled_trainloader = get_unlabeled_set(batch_size=batch_size)

In [24]:
# sample, target, road_image, extra = iter(labeled_trainloader).next()
# print(torch.stack(sample).shape)

image, camera_index = iter(unlabeled_trainloader).next()
print(image.shape)

torch.Size([3, 3, 256, 306])


# Model

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
device

device(type='cpu')

### Hyperparameters

In [26]:
epochs = 1
hidden_size = 1024

model = Prototype(hidden_dim=hidden_size)

if torch.cuda.is_available():
    model = torch.nn.DataParallel(model)
    assert batch_size >= torch.cuda.device_count()

model = model.to(device)
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters())

In [17]:
model.train()

for epoch in range(epochs):
    loss = 0.0
    
    max_batches = len(unlabeled_trainloader)
    for idx, (images, camera_index) in enumerate(unlabeled_trainloader):
        optimizer.zero_grad()
        
        images = images.to(device)
        reconstructions = model(images)
        loss = criterion(reconstructions, images)
        
        loss.backward()
        optimizer.step()
        
        if idx % 1000 == 0:
            print('[', epoch, '|', idx ,'/', max_batches, ']', 'loss:', loss.item())

[ 0 | 0 / 26712 ] loss: 0.5756482481956482
[ 0 | 1 / 26712 ] loss: 0.652309000492096
[ 0 | 2 / 26712 ] loss: 0.5202575922012329
[ 0 | 3 / 26712 ] loss: 0.6401046514511108
[ 0 | 4 / 26712 ] loss: 0.645193338394165
[ 0 | 5 / 26712 ] loss: 0.6373991966247559
[ 0 | 6 / 26712 ] loss: 0.6325534582138062
[ 0 | 7 / 26712 ] loss: 0.5634716153144836
[ 0 | 8 / 26712 ] loss: 0.604899525642395
[ 0 | 9 / 26712 ] loss: 0.6308258175849915
[ 0 | 10 / 26712 ] loss: 0.6094906330108643
[ 0 | 11 / 26712 ] loss: 0.6008045673370361
[ 0 | 12 / 26712 ] loss: 0.5547239184379578
[ 0 | 13 / 26712 ] loss: 0.6735868453979492
[ 0 | 14 / 26712 ] loss: 0.5667365789413452
[ 0 | 15 / 26712 ] loss: 0.5744085907936096
[ 0 | 16 / 26712 ] loss: 0.5768027305603027
[ 0 | 17 / 26712 ] loss: 0.5797910094261169
[ 0 | 18 / 26712 ] loss: 0.6291548013687134
[ 0 | 19 / 26712 ] loss: 0.5445032715797424
[ 0 | 20 / 26712 ] loss: 0.542401909828186
[ 0 | 21 / 26712 ] loss: 0.6173267960548401
[ 0 | 22 / 26712 ] loss: 0.6424456834793091
[ 

KeyboardInterrupt: 